In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
dat = pd.read_csv(r'C:\Users\bovas\Desktop\Main Project\Aroma Detection\garlic.csv')

In [2]:
dat.head()

,timestamp,voc1,no2,eth,co,tag
0,2236,3.17,2.64,3.31,3.54,garlic
1,2486,3.17,2.64,3.31,3.55,garlic
2,2736,3.18,2.64,3.31,3.55,garlic
3,2986,3.18,2.65,3.31,3.55,garlic
4,3236,3.18,2.64,3.31,3.56,garlic


In [3]:
dat.drop(['timestamp'],axis=1, inplace=True)

In [4]:
dat

,voc1,no2,eth,co,tag
0,3.17,2.64,3.31,3.54,garlic
1,3.17,2.64,3.31,3.55,garlic
2,3.18,2.64,3.31,3.55,garlic
3,3.18,2.65,3.31,3.55,garlic
4,3.18,2.64,3.31,3.56,garlic
...,...,...,...,...,...
3095,0.84,1.10,0.64,1.95,background
3096,0.84,1.10,0.65,1.95,background
3097,0.84,1.10,0.64,1.95,background
3098,0.84,1.10,0.65,1.95,background


In [5]:
X, y = dat[['voc1','no2','eth','co']], dat['tag']

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=50, batch_size=8, validation_split=0.2)

Epoch 1/50
310/310 [==============================] - 1s 3ms/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 1.8304 - val_accuracy: 0.4258
Epoch 2/50
310/310 [==============================] - 1s 2ms/step - loss: 2.5756e-05 - accuracy: 1.0000 - val_loss: 2.1053 - val_accuracy: 0.4258
Epoch 3/50
310/310 [==============================] - 1s 2ms/step - loss: 7.9061e-06 - accuracy: 1.0000 - val_loss: 2.2648 - val_accuracy: 0.4258
Epoch 4/50
310/310 [==============================] - 1s 2ms/step - loss: 3.7969e-06 - accuracy: 1.0000 - val_loss: 2.3792 - val_accuracy: 0.4258
Epoch 5/50
310/310 [==============================] - 0s 2ms/step - loss: 2.1984e-06 - accuracy: 1.0000 - val_loss: 2.4695 - val_accuracy: 0.4258
Epoch 6/50
310/310 [==============================] - 0s 2ms/step - loss: 1.4120e-06 - accuracy: 1.0000 - val_loss: 2.5452 - val_accuracy: 0.4258
Epoch 7/50
310/310 [==============================] - 1s 2ms/step - loss: 9.6859e-07 - accuracy: 1.0000 - val_loss: 2.6117 - val

In [7]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

20/20 [==============================] - 0s 1ms/step - loss: 0.1788 - accuracy: 0.8871
Test Loss: 0.1788, Test Accuracy: 0.8871


In [8]:
import numpy as np
inputs = np.array([[1,1,0.5,2]])
scaled_input = scaler.transform(inputs)
predictions = model.predict(scaled_input)
pred_labels = (predictions > 0.5).astype(int)
decoded_labels = label_encoder.inverse_transform(pred_labels.flatten())

print("Predicted Labels:")
print(decoded_labels)


1/1 [==============================] - 0s 88ms/step
Predicted Labels:
['garlic']


c:\Users\bovas\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [9]:
model.save(r"C:\Users\bovas\Desktop\Main Project\Aroma Detection\ann_model")

INFO:tensorflow:Assets written to: C:\Users\bovas\Desktop\Main Project\Aroma Detection\ann_model\assets


INFO:tensorflow:Assets written to: C:\Users\bovas\Desktop\Main Project\Aroma Detection\ann_model\assets


In [6]:
import serial
import numpy as np
from tensorflow.keras.models import load_model


X, y = dat[['voc1','no2','eth','co']], dat['tag']

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)


# Load the pre-trained model
model = load_model(r"C:\Users\bovas\Desktop\Main Project\Aroma Detection\ann_model")

# Set the serial port (adjust as needed)
ser = serial.Serial('COM6', 115200)  # Change 'COM3' to your Arduino port

while True:
    arduino_data = ser.readline().decode('utf-8').strip().split(',')

    if arduino_data:
        sensor_values = [float(value) for value in arduino_data]
        sensor_values = np.expand_dims(sensor_values, axis=0)

        # Make predictions
        predictions = model.predict(sensor_values)
        print(predictions)
     
        pred_labels = (predictions > 0.5).astype(int)
        decoded_labels = label_encoder.inverse_transform(pred_labels.flatten())

        print("Predicted Labels:")
        print(decoded_labels)

1/1 [==============================] - 0s 178ms/step
[[0.99578875]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 33ms/step
[[0.99578875]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 32ms/step
[[0.99584085]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 16ms/step
[[0.9959082]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 25ms/step
[[0.9958355]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 29ms/step
[[0.9958869]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 27ms/step
[[0.9959082]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 17ms/step
[[0.9958355]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 36ms/step
[[0.99578875]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 28ms/step
[[0.99578875]]
Predicted Labels:
['garlic']
1/1 [==============================] - 0s 63

KeyboardInterrupt: 